## CNN network for sentiment analysis
network inspired by this [post](http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/)

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

## Parameters

In [2]:
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 1.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=1.0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=10
NUM_FILTERS=128
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos



## Data Loading from sentiment RNN

In [14]:
x_train = np.load('../deep-learning/sentiment-rnn/train_x.npy')
x_dev = np.load('../deep-learning/sentiment-rnn/val_x.npy')
x_test = np.load('../deep-learning/sentiment-rnn/test_x.npy')
y_train = np.load('../deep-learning/sentiment-rnn/train_y.npy')
y_dev = np.load('../deep-learning/sentiment-rnn/val_y.npy')
y_test = np.load('../deep-learning/sentiment-rnn/test_y.npy')

In [29]:
y_test = np.argmax(y_test, axis=1)
print(y_test)

[1 0 1 ..., 0 1 0]


## Training

In [20]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=74073,#len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2017-08-02T10:27:43.752123: step 98, loss 2.29522, acc 0.734375
2017-08-02T10:27:44.330181: step 99, loss 2.74733, acc 0.5
2017-08-02T10:27:44.907239: step 100, loss 3.05019, acc 0.421875

Evaluation:
2017-08-02T10:27:48.579606: step 100, loss 1.6785, acc 0.6696

Saved model checkpoint to C:\Users\E411208\Documents\Python Scripts\cnn-text-classification-tf\runs\1501662405\checkpoints\model-100

2017-08-02T10:27:51.792927: step 101, loss 2.29843, acc 0.5625
2017-08-02T10:27:52.373986: step 102, loss 2.73785, acc 0.5
2017-08-02T10:27:52.958044: step 103, loss 2.60496, acc 0.4375
2017-08-02T10:27:53.509099: step 104, loss 2.65396, acc 0.4375
2017-08-02T10:27:54.059154: step 105, loss 2.84236, acc 0.5
2017-08-02T10:27:54.637212: step 106, loss 3.26774, acc 0.4375
2017-08-02T10:27:55.194268: step 107, loss 2.53183, acc 0.53125
2017-08-02T10:27:55.764325: step 108, loss 2.51604, acc 0.515625
2017-08-02T10:27:56.309379: step 109, loss 2.48767, acc 0.546875
2017-08-02T10:27:56.863434: step 110

2017-08-02T10:29:06.615409: step 220, loss 1.26836, acc 0.5625
2017-08-02T10:29:07.175465: step 221, loss 1.2504, acc 0.53125
2017-08-02T10:29:07.819529: step 222, loss 1.3062, acc 0.484375
2017-08-02T10:29:08.397587: step 223, loss 1.20091, acc 0.59375
2017-08-02T10:29:08.956643: step 224, loss 1.38064, acc 0.515625
2017-08-02T10:29:09.513699: step 225, loss 1.43573, acc 0.546875
2017-08-02T10:29:10.070754: step 226, loss 1.27575, acc 0.5625
2017-08-02T10:29:10.622810: step 227, loss 1.33985, acc 0.453125
2017-08-02T10:29:11.176865: step 228, loss 1.29191, acc 0.578125
2017-08-02T10:29:11.730920: step 229, loss 1.26866, acc 0.5625
2017-08-02T10:29:12.288976: step 230, loss 1.15978, acc 0.671875
2017-08-02T10:29:12.857033: step 231, loss 1.31217, acc 0.5625
2017-08-02T10:29:13.423090: step 232, loss 1.22366, acc 0.609375
2017-08-02T10:29:13.981145: step 233, loss 1.19563, acc 0.65625
2017-08-02T10:29:14.538201: step 234, loss 1.3383, acc 0.546875
2017-08-02T10:29:15.098257: step 235, l

2017-08-02T10:30:24.742221: step 345, loss 0.872481, acc 0.59375
2017-08-02T10:30:25.291276: step 346, loss 0.923791, acc 0.546875
2017-08-02T10:30:25.848331: step 347, loss 0.844692, acc 0.671875
2017-08-02T10:30:26.405387: step 348, loss 0.994808, acc 0.546875
2017-08-02T10:30:26.959443: step 349, loss 0.897281, acc 0.59375
2017-08-02T10:30:27.536500: step 350, loss 0.965204, acc 0.53125
2017-08-02T10:30:28.098556: step 351, loss 0.86976, acc 0.59375
2017-08-02T10:30:28.653612: step 352, loss 0.991244, acc 0.53125
2017-08-02T10:30:29.209668: step 353, loss 0.875825, acc 0.640625
2017-08-02T10:30:29.772724: step 354, loss 0.835198, acc 0.625
2017-08-02T10:30:30.323779: step 355, loss 0.858524, acc 0.625
2017-08-02T10:30:30.881835: step 356, loss 0.813104, acc 0.65625
2017-08-02T10:30:31.431890: step 357, loss 0.845312, acc 0.578125
2017-08-02T10:30:31.989946: step 358, loss 0.902421, acc 0.59375
2017-08-02T10:30:32.548001: step 359, loss 0.72644, acc 0.75
2017-08-02T10:30:33.107057: s

2017-08-02T10:31:42.141960: step 469, loss 0.650981, acc 0.71875
2017-08-02T10:31:42.694015: step 470, loss 0.779108, acc 0.625
2017-08-02T10:31:43.265072: step 471, loss 0.664454, acc 0.734375
2017-08-02T10:31:43.819128: step 472, loss 0.797251, acc 0.609375
2017-08-02T10:31:44.375183: step 473, loss 0.708133, acc 0.71875
2017-08-02T10:31:44.935239: step 474, loss 0.727648, acc 0.640625
2017-08-02T10:31:45.490295: step 475, loss 0.718257, acc 0.625
2017-08-02T10:31:46.049351: step 476, loss 0.776705, acc 0.640625
2017-08-02T10:31:46.616407: step 477, loss 0.701417, acc 0.671875
2017-08-02T10:31:47.171463: step 478, loss 0.74147, acc 0.640625
2017-08-02T10:31:47.733519: step 479, loss 0.816209, acc 0.484375
2017-08-02T10:31:48.286574: step 480, loss 0.757392, acc 0.5625
2017-08-02T10:31:48.839630: step 481, loss 0.690749, acc 0.65625
2017-08-02T10:31:49.398686: step 482, loss 0.78349, acc 0.59375
2017-08-02T10:31:49.957742: step 483, loss 0.78944, acc 0.625
2017-08-02T10:31:50.513797: 

...

2017-08-02T10:54:28.259558: step 2648, loss 0.448813, acc 0.8125
2017-08-02T10:54:28.815614: step 2649, loss 0.235606, acc 0.953125
2017-08-02T10:54:29.367669: step 2650, loss 0.422457, acc 0.84375
2017-08-02T10:54:29.934726: step 2651, loss 0.27401, acc 0.90625
2017-08-02T10:54:30.497782: step 2652, loss 0.324158, acc 0.890625
2017-08-02T10:54:31.049837: step 2653, loss 0.280046, acc 0.875
2017-08-02T10:54:31.608893: step 2654, loss 0.327412, acc 0.859375
2017-08-02T10:54:32.172949: step 2655, loss 0.326316, acc 0.90625
2017-08-02T10:54:32.731005: step 2656, loss 0.392328, acc 0.84375
2017-08-02T10:54:33.294062: step 2657, loss 0.258204, acc 0.90625
2017-08-02T10:54:33.860118: step 2658, loss 0.263745, acc 0.90625
2017-08-02T10:54:34.418174: step 2659, loss 0.245363, acc 0.890625
2017-08-02T10:54:34.981230: step 2660, loss 0.211496, acc 0.9375
2017-08-02T10:54:35.550287: step 2661, loss 0.305966, acc 0.890625
2017-08-02T10:54:36.108343: step 2662, loss 0.196211, acc 0.9375
2017-08-02T

2017-08-02T10:55:42.337965: step 2769, loss 0.37555, acc 0.890625
2017-08-02T10:55:42.901022: step 2770, loss 0.342435, acc 0.875
2017-08-02T10:55:43.459077: step 2771, loss 0.204268, acc 0.921875
2017-08-02T10:55:44.016133: step 2772, loss 0.392493, acc 0.828125
2017-08-02T10:55:44.574189: step 2773, loss 0.411069, acc 0.875
2017-08-02T10:55:45.129244: step 2774, loss 0.369317, acc 0.859375
2017-08-02T10:55:45.686300: step 2775, loss 0.320571, acc 0.859375
2017-08-02T10:55:46.239355: step 2776, loss 0.333833, acc 0.890625
2017-08-02T10:55:46.801412: step 2777, loss 0.259121, acc 0.9375
2017-08-02T10:55:47.362468: step 2778, loss 0.261686, acc 0.921875
2017-08-02T10:55:47.932525: step 2779, loss 0.228704, acc 0.9375
2017-08-02T10:55:48.489580: step 2780, loss 0.386498, acc 0.875
2017-08-02T10:55:49.045636: step 2781, loss 0.249533, acc 0.890625
2017-08-02T10:55:49.599691: step 2782, loss 0.372877, acc 0.859375
2017-08-02T10:55:50.158747: step 2783, loss 0.207996, acc 0.9375
2017-08-02T

2017-08-02T10:56:55.750306: step 2890, loss 0.173207, acc 0.953125
2017-08-02T10:56:56.311362: step 2891, loss 0.172783, acc 0.96875
2017-08-02T10:56:56.874418: step 2892, loss 0.131257, acc 1
2017-08-02T10:56:57.431474: step 2893, loss 0.300649, acc 0.890625
2017-08-02T10:56:57.986529: step 2894, loss 0.271783, acc 0.890625
2017-08-02T10:56:58.544585: step 2895, loss 0.29316, acc 0.875
2017-08-02T10:56:59.105641: step 2896, loss 0.212363, acc 0.953125
2017-08-02T10:56:59.662697: step 2897, loss 0.225315, acc 0.953125
2017-08-02T10:57:00.219753: step 2898, loss 0.242321, acc 0.953125
2017-08-02T10:57:00.776808: step 2899, loss 0.240791, acc 0.90625
2017-08-02T10:57:01.333864: step 2900, loss 0.23962, acc 0.96875

Evaluation:
2017-08-02T10:57:04.984229: step 2900, loss 0.424681, acc 0.8316

Saved model checkpoint to C:\Users\E411208\Documents\Python Scripts\cnn-text-classification-tf\runs\1501662405\checkpoints\model-2900

2017-08-02T10:57:07.561487: step 2901, loss 0.259371, acc 0.9687

2017-08-02T10:58:13.978128: step 3008, loss 0.28372, acc 0.9375
2017-08-02T10:58:14.538184: step 3009, loss 0.273341, acc 0.859375
2017-08-02T10:58:15.115241: step 3010, loss 0.230825, acc 0.90625
2017-08-02T10:58:15.666297: step 3011, loss 0.213888, acc 0.953125
2017-08-02T10:58:16.217352: step 3012, loss 0.27802, acc 0.921875
2017-08-02T10:58:16.770407: step 3013, loss 0.196034, acc 0.9375
2017-08-02T10:58:17.335463: step 3014, loss 0.231401, acc 0.90625
2017-08-02T10:58:17.900520: step 3015, loss 0.311814, acc 0.921875
2017-08-02T10:58:18.454575: step 3016, loss 0.311643, acc 0.859375
2017-08-02T10:58:19.012631: step 3017, loss 0.273509, acc 0.921875
2017-08-02T10:58:19.570687: step 3018, loss 0.22154, acc 0.921875
2017-08-02T10:58:20.130743: step 3019, loss 0.303899, acc 0.890625
2017-08-02T10:58:20.706801: step 3020, loss 0.189053, acc 0.9375
2017-08-02T10:58:21.258856: step 3021, loss 0.223729, acc 0.90625
2017-08-02T10:58:21.821912: step 3022, loss 0.343081, acc 0.890625
2017-08

2017-08-02T10:59:27.569486: step 3129, loss 0.270102, acc 0.890625
2017-08-02T10:59:27.982527: step 3130, loss 0.310269, acc 0.9375


## Evaluation

In [30]:
checkpoint_file = tf.train.latest_checkpoint(os.path.join('runs/1501662405', "checkpoints"))
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

INFO:tensorflow:Restoring parameters from C:\Users\E411208\Documents\Python Scripts\cnn-text-classification-tf\runs\1501662405\checkpoints\model-3100
Total number of test examples: 2500
Accuracy: 0.8228
